In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
# Step 1: Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar100.load_data()


In [3]:
# Step 2: Preprocess the CIFAR-100 dataset
x_train = x_train.astype('float32') / 255.0  # Normalize the images to [0, 1]
x_test = x_test.astype('float32') / 255.0

In [4]:
# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 100)
y_test = tf.keras.utils.to_categorical(y_test, 100)


## Using Resnet50

In [5]:
# Step 3: Load the pretrained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

In [6]:
# Step 4: Add custom layers on top of ResNet50
x = layers.GlobalAveragePooling2D()(base_model.output)  # Pooling layer
x = layers.Dense(512, activation='relu')(x)             # Fully connected layer
x = layers.Dropout(0.5)(x)                             # Dropout for regularization
predictions = layers.Dense(100, activation='softmax')(x)  # Output layer (100 classes for CIFAR-100)

In [7]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
# Step 5: Freeze the base model layers for transfer learning
for layer in base_model.layers:
    layer.trainable = False

In [9]:
# Step 6: Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Step 7: Train the model
history = model.fit(x_train, 
                    y_train, 
                    epochs=30,
                    batch_size=64, 
                    validation_data=(x_test, y_test))

Epoch 1/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 55s 64ms/step - accuracy: 0.0098 - loss: 4.6750 - val_accuracy: 0.0100 - val_loss: 4.6052
Epoch 2/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 48s 62ms/step - accuracy: 0.0097 - loss: 4.6055 - val_accuracy: 0.0100 - val_loss: 4.6052
Epoch 3/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 48s 61ms/step - accuracy: 0.0109 - loss: 4.5984 - val_accuracy: 0.0207 - val_loss: 4.5550
Epoch 4/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 47s 61ms/step - accuracy: 0.0145 - loss: 4.5774 - val_accuracy: 0.0215 - val_loss: 4.5513
Epoch 5/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 48s 61ms/step - accuracy: 0.0147 - loss: 4.5727 - val_accuracy: 0.0208 - val_loss: 4.5548
Epoch 6/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 48s 61ms/step - accuracy: 0.0143 - loss: 4.5721 - val_accuracy: 0.0211 - val_loss: 4.5475
Epoch 7/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 48s 62ms/step - accuracy: 0.0152 - loss: 4.5688 - val_accuracy: 0.0204 - val_loss: 4.5392
Epoch 8/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 47s 60ms/step - accuracy: 0.0177 - loss: 4.5616 - 

In [ ]:
# Step 8: Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")